In [24]:
import joblib
import pandas as pd
import numpy as np
from preprocessing import preprocessing, one_hot_encoder
from feature_engineering import create_feat

from utils import read_yaml

PREPROCESSING_CONFIG_PATH = "../config/preprocessing_config.yaml"
PREDICT_CONFIG_PATH = "../config/predict_config.yaml"

params_prep = read_yaml(PREPROCESSING_CONFIG_PATH)
params = read_yaml(PREDICT_CONFIG_PATH)

normalizer = joblib.load("../output/normalizer.pkl")
ohe = joblib.load("../output/onehotencoder.pkl")
model = joblib.load('../output/model_name.pkl')
estimator = joblib.load('../output/best_estimator.pkl')

def construct(test_predict, params_prep):    
    df_test = pd.DataFrame(test_predict).astype(int)
    feat = create_feat(df_test)
    df_normalizer = preprocessing(feat, params_prep, state=normalizer)[0]
    df_ohe = ohe.transform(feat[['SEX', 'MARRIAGE']])
    col = ohe.get_feature_names_out()
    df_ohe = pd.DataFrame(df_ohe, columns=col)
    result = pd.merge(df_ohe, df_normalizer, how="left")
    return result

def main_predict(prediksian):
    to_predict_model = model.predict(prediksian) #sek salah
    if to_predict_model == [0]:
        print("Non Deafult")
    else:
        print("Default")

if __name__ == "__main__":
    n_data = int(input(f"Input data (enter int value): "))
    data_predict = {}
    for i in range(n_data):
        for i in params["x_col"]:
            if i in data_predict:
                data_predict[i].append(input(f"Input {i}: "))
            else:
                data_predict[i] = [input(f"Input {i}: ")]
    to_pred = construct(data_predict, params_prep)
    main_predict(to_pred)

C:\Users\yuha bach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- MARRIAGE_2
- MARRIAGE_3
- SEX_2

  warnings.warn(message, FutureWarning)


ValueError: X has 30 features, but MLPClassifier is expecting 33 features as input.

In [ ]:
construct(data_predict, params_prep)

In [72]:
from preprocessing import preprocessing, one_hot_encoder

test_predict = data_predict
df_test = pd.DataFrame(test_predict).astype(int)
feat = create_feat(df_test)
df_normalizer = preprocessing(feat, params_prep, state=normalizer)[0]
to_ohe = one_hot_encoder(feat[['SEX', 'MARRIAGE']], params_prep, state=ohe)
col = ohe.get_feature_names_out()
df_ohe = pd.DataFrame(to_ohe, columns=col)
result = pd.merge(df_ohe, df_normalizer, how="left")

AttributeError: 'dict' object has no attribute 'index'

In [68]:
feat

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,AgeBin,Closeness_6,Closeness_5,Closeness_4,Closeness_3,Closeness_2,Closeness_1
0,123,10000,1,2,1,33,0,1,2,1,...,500,400,200,2,0.98,0.96,0.5,0.991,0.993,0.1


In [71]:
one_hot_encoder(feat[['SEX', 'MARRIAGE']], params_prep, state='normalizer')

AttributeError: 'dict' object has no attribute 'index'

In [64]:
feat[['SEX', 'MARRIAGE']].index

RangeIndex(start=0, stop=1, step=1)

In [40]:
main_predict(result)

C:\Users\yuha bach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- MARRIAGE_2
- MARRIAGE_3
- SEX_2

  warnings.warn(message, FutureWarning)


ValueError: X has 30 features, but MLPClassifier is expecting 33 features as input.

In [37]:
data_predict

{'ID': ['123'],
 'LIMIT_BAL': ['10000'],
 'SEX': ['1'],
 'EDUCATION': ['2'],
 'MARRIAGE': ['1'],
 'AGE': ['33'],
 'PAY_0': ['0'],
 'PAY_2': ['1'],
 'PAY_3': ['2'],
 'PAY_4': ['1'],
 'PAY_5': ['0'],
 'PAY_6': ['-1'],
 'BILL_AMT1': ['9000'],
 'BILL_AMT2': ['70'],
 'BILL_AMT3': ['90'],
 'BILL_AMT4': ['5000'],
 'BILL_AMT5': ['400'],
 'BILL_AMT6': ['200'],
 'PAY_AMT1': ['90'],
 'PAY_AMT2': ['800'],
 'PAY_AMT3': ['60'],
 'PAY_AMT4': ['500'],
 'PAY_AMT5': ['400'],
 'PAY_AMT6': ['200']}

In [41]:
ohe

OneHotEncoder(handle_unknown='ignore', sparse=False)